In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import tree
import mord 
from sklearn import linear_model, metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [73]:
# DOG DATA WRANGLING
train = pd.read_csv('train.csv')
train.dropna(inplace=True)
train_dogs = train[train['Type'] == 1]
train_dogs_adoption_speed_old = train_dogs['AdoptionSpeed']
train_type_id = train_dogs[['Type', 'PetID']]

train_dogs_adoption_speed = []
for i in train_dogs_adoption_speed_old:
    if i <= 3:
        train_dogs_adoption_speed.append(0)
    else: 
        train_dogs_adoption_speed.append(1)

data_dogs = {}
train_dogs_magnitudes = [];
train_dogs_scores = [];
test_dogs_magnitudes = [];
test_dogs_scores = [];

# Clean up json files just for dogs
for root, dirs, files in os.walk("train_sentiment"):
    for filename in files:
        loc = train_type_id.loc[train_type_id['PetID'] == filename[0:len(filename)-5]]
        if len(loc.index) > 0 and filename.endswith(".json"):
            data_file_name = 'train_sentiment/' + filename
            data_dogs = json.load(open(data_file_name, 'r'))
            train_dogs_magnitudes.append(data_dogs["documentSentiment"]["magnitude"])
            train_dogs_scores.append(data_dogs["documentSentiment"]["score"])
            
df_train_dogs_magnitudes = pd.DataFrame(train_dogs_magnitudes)
df_train_dogs_scores = pd.DataFrame(train_dogs_scores)
df_train_dogs_adoption_speed = pd.DataFrame(train_dogs_adoption_speed)

horizontal_dogs_train_df = pd.concat([df_train_dogs_magnitudes, df_train_dogs_scores, df_train_dogs_adoption_speed], axis=1)
horizontal_dogs_train_df.columns = ['train_dogs_magnitudes', 'train_dogs_scores', 'train_dogs_adoption_speed']
horizontal_dogs_train_df.dropna(inplace = True)

features_train_dogs = ['train_dogs_magnitudes']
# Decision trees, SVM, Linear SVM, Nearest Neighbors, Logistic Regression

In [7]:
# CAT DATA WRANGLING
train = pd.read_csv('train.csv')
train.dropna(inplace=True)
train_cats = train[train['Type'] == 2]
train_cats_adoption_speed_old = train_cats['AdoptionSpeed']
train_type_cats_id = train_cats[['Type', 'PetID']]

train_cats_adoption_speed = []
for i in train_cats_adoption_speed_old:
    if i <= 3:
        train_cats_adoption_speed.append(0)
    else:
        train_cats_adoption_speed.append(1)

data_cats = {}
train_cats_magnitudes = [];
train_cats_scores = [];
test_cats_magnitudes = [];
test_cats_scores = [];

# Clean up json files just for dogs
for root, dirs, files in os.walk("train_sentiment"):
    for filename in files:
        loc = train_type_cats_id.loc[train_type_cats_id['PetID'] == filename[0:len(filename)-5]]
        if len(loc.index) > 0 and filename.endswith(".json"):
            data_file_name = 'train_sentiment/' + filename;
            data_cats = json.load(open(data_file_name, 'r'))
            train_cats_magnitudes.append(data_cats["documentSentiment"]["magnitude"])
            train_cats_scores.append(data_cats["documentSentiment"]["score"])
            
df_train_cats_magnitudes = pd.DataFrame(train_cats_magnitudes)
df_train_cats_scores = pd.DataFrame(train_cats_scores)
df_train_cats_adoption_speed = pd.DataFrame(train_cats_adoption_speed)

horizontal_cats_train_df = pd.concat([df_train_cats_magnitudes, df_train_cats_scores, df_train_cats_adoption_speed], axis=1)
horizontal_cats_train_df.columns = ['train_cats_magnitudes', 'train_cats_scores', 'train_cats_adoption_speed']
horizontal_cats_train_df.dropna(inplace = True)

features_train_cats = ['train_cats_scores']

In [128]:
# Logistic Regression on Dog Data
X = horizontal_dogs_train_df[features_train_dogs]
y = horizontal_dogs_train_df['train_dogs_adoption_speed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=777)
print(pd.DataFrame(y_train)['train_dogs_adoption_speed'].value_counts())

parameters = {'C':[1, 5, 10]}
lg = LogisticRegression(penalty='l2', solver='newton-cg', random_state=777, max_iter=10000, tol=10)
clf = GridSearchCV(lg, parameters, cv=5, refit='True')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("The score of the dogs model is (TRAIN): %f" % clf.score(X_train, y_train))
print("-------------")
print("The score of the dogs model is (TEST): %f" % clf.score(X_test, y_test))
print("-------------")
print("The best params for logistic regression on dogs is: %s " % clf.best_params_)
print("-------------")

print("The F1 score for the dogs model is (TEST): %f" % f1_score(y_test, y_pred))


0    2664
1    1055
Name: train_dogs_adoption_speed, dtype: int64
The score of the dogs model is (TRAIN): 0.716322
-------------
The score of the dogs model is (TEST): 0.707527
-------------
The best params for logistic regression on dogs is: {'C': 1} 
-------------
The F1 score for the dogs model is (TEST): 0.000000


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [129]:
# Logistic Regression on Cat Data
X = horizontal_cats_train_df.loc[:, features_train_cats]
y = horizontal_cats_train_df['train_cats_adoption_speed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print(pd.DataFrame(y_train)['train_cats_adoption_speed'].value_counts())

parameters = {'C':[1, 5, 10]}
lg = LogisticRegression(penalty='l2', solver='newton-cg', random_state=777, max_iter=10000, tol=10)
clf = GridSearchCV(lg, parameters, cv=5)
clf.fit(X_train, y_train)

print("The score of the cats model is (TRAIN): %f" % clf.score(X_train, y_train))
print("-------------")
print("The score of the cats model is (TEST): %f" % clf.score(X_test, y_test))
print("-------------")
print("The best params for logistic regression on cats is: %s " % clf.best_params_)
print("-------------")
y_pred = clf.predict(X_test)
y_pred_df = pd.DataFrame(y_pred);
y_test_df = pd.DataFrame(y_test);

print("The F1 score for the dogs model is (TEST): %f" % f1_score(y_test, y_pred, average="weighted"))

0    2590
1     875
Name: train_cats_adoption_speed, dtype: int64
The score of the cats model is (TRAIN): 0.747475
-------------
The score of the cats model is (TEST): 0.739507
-------------
The best params for logistic regression on cats is: {'C': 1} 
-------------
The F1 score for the dogs model is (TEST): 0.628765


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [130]:
# Decision Trees on Dog Data
X = horizontal_dogs_train_df.loc[:, features_train_dogs]
y = horizontal_dogs_train_df['train_dogs_adoption_speed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print(pd.DataFrame(y_train)['train_dogs_adoption_speed'].value_counts())

parameters = {'max_depth': np.arange(3, 10)}
dt = DecisionTreeClassifier(random_state=0)
clf = GridSearchCV(dt, parameters, cv=5)
clf.fit(X_train, y_train)

print("The score of the dogs model is (TRAIN): %f" % clf.score(X_train, y_train))
print("-------------")
print("The score of the dogs model is (TEST): %f" % clf.score(X_test, y_test))
print("-------------")
print("The best params for logistic regression on dogs is: %s " % clf.best_params_)
print("-------------")
y_pred = clf.predict(X_test)

print("The F1 score for the dogs model is (TEST): %f" % f1_score(y_test, y_pred, average="weighted"))

0    3161
1    1302
Name: train_dogs_adoption_speed, dtype: int64
The score of the dogs model is (TRAIN): 0.708940
-------------
The score of the dogs model is (TEST): 0.717406
-------------
The best params for logistic regression on dogs is: {'max_depth': 3} 
-------------
The F1 score for the dogs model is (TEST): 0.599983


In [131]:
# Decision Trees on CAT Data
X = horizontal_cats_train_df.loc[:, features_train_cats]
y = horizontal_cats_train_df['train_cats_adoption_speed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print(pd.DataFrame(y_train)['train_cats_adoption_speed'].value_counts())

parameters = {'C':[1, 5, 10]}
parameters = {'max_depth': np.arange(3, 10)}
dt = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

print("The score of the cats model is (TRAIN): %f" % clf.score(X_train, y_train))
print("-------------")
print("The score of the cats model is (TEST): %f" % clf.score(X_test, y_test))
print("-------------")
print("The best params for logistic regression on cats is: %s " % clf.best_params_)
print("-------------")
y_pred = clf.predict(X_test)

print("The F1 score for the dogs model is (TEST): %f" % f1_score(y_test, y_pred, average='weighted'))

0    2590
1     875
Name: train_cats_adoption_speed, dtype: int64
The score of the cats model is (TRAIN): 0.747763
-------------
The score of the cats model is (TEST): 0.739507
-------------
The best params for logistic regression on cats is: {'max_depth': 3} 
-------------
The F1 score for the dogs model is (TEST): 0.628765


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [132]:
# K-Nearest Neighbors on DOG Data
X = horizontal_dogs_train_df.loc[:, features_train_dogs]
y = horizontal_dogs_train_df['train_dogs_adoption_speed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print(pd.DataFrame(y_train)['train_dogs_adoption_speed'].value_counts())

neigh = KNeighborsClassifier()
parameters = {'n_neighbors': [3, 4, 5]}
clf = GridSearchCV(neigh, parameters, cv=5)
clf.fit(X_train, y_train)
print("-------------")
print("The score of the dogs model is (TRAIN): %f" % clf.score(X_train, y_train))
print("-------------")
print("The score of the dogs model is (TEST): %f" % clf.score(X_test, y_test))
print("-------------")
print("The best params for logistic regression on dogs is: %s " % clf.best_params_)
print("-------------")
y_pred = clf.predict(X_test)

print("The F1 score for the dogs model is (TEST): %f" % f1_score(y_test, y_pred, average="weighted"))

0    3161
1    1302
Name: train_dogs_adoption_speed, dtype: int64
-------------
The score of the dogs model is (TRAIN): 0.693256
-------------
The score of the dogs model is (TEST): 0.698925
-------------
The best params for logistic regression on dogs is: {'n_neighbors': 4} 
-------------
The F1 score for the dogs model is (TEST): 0.603421


In [133]:
# K-Nearest Neighbors on CAT Data
X = horizontal_cats_train_df.loc[:, features_train_cats]
y = horizontal_cats_train_df['train_cats_adoption_speed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
neigh = KNeighborsClassifier()
parameters = {'n_neighbors': [3, 4, 5]}
clf = GridSearchCV(neigh, parameters, cv=5)
clf.fit(X_train, y_train)

print("-------------")
print("The score of the cats model is (TRAIN): %f" % clf.score(X_train, y_train))
print("-------------")
print("The score of the cats model is (TEST): %f" % clf.score(X_test, y_test))
print("-------------")
print("The best params for logistic regression on cats is: %s " % clf.best_params_)
print("-------------")
y_pred = clf.predict(X_test)

print("The F1 score for the dogs model is (TEST): %f" % f1_score(y_test, y_pred, average="weighted"))

-------------
The score of the cats model is (TRAIN): 0.608658
-------------
The score of the cats model is (TEST): 0.610125
-------------
The best params for logistic regression on cats is: {'n_neighbors': 4} 
-------------
The F1 score for the dogs model is (TEST): 0.615983


In [138]:
# LinearSVC on DOG Data
X = horizontal_dogs_train_df.loc[:, features_train_dogs]
y = horizontal_dogs_train_df['train_dogs_adoption_speed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
svc = LinearSVC(random_state=0, tol=1e-5, max_iter=1000)
parameters = {'C':[0.001, 0.01, 0.1, 1, 10]}
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train, y_train)
print("-------------")
print("The score of the dogs model is (TRAIN): %f" % clf.score(X_train, y_train))
print("-------------")
print("The score of the dogs model is (TEST): %f" % clf.score(X_test, y_test))
print("-------------")
print("The best params for logistic regression on dogs is: %s " % clf.best_params_)
print("-------------")
y_pred = clf.predict(X_test)

print("The F1 score for the dogs model is (TEST): %f" % f1_score(y_test, y_pred, average="weighted"))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Version

-------------
The score of the dogs model is (TRAIN): 0.708268
-------------
The score of the dogs model is (TEST): 0.717406
-------------
The best params for logistic regression on dogs is: {'C': 0.001} 
-------------
The F1 score for the dogs model is (TEST): 0.599359


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [139]:
# LinearSVC on CAT Data
X = horizontal_cats_train_df.loc[:, features_train_cats]
y = horizontal_cats_train_df['train_cats_adoption_speed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
svc = LinearSVC(random_state=0, tol=1e-5)
parameters = {'C': [0.001, 0.01, 0.1, 1, 10]}
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train, y_train)

print("-------------")
print("The score of the cats model is (TRAIN): %f" % clf.score(X_train, y_train))
print("-------------")
print("The score of the cats model is (TEST): %f" % clf.score(X_test, y_test))
print("-------------")
print("The best params for logistic regression on cats is: %s " % clf.best_params_)
print("-------------")
y_pred = clf.predict(X_test)

print("The F1 score for the dogs model is (TEST): %f" % f1_score(y_test, y_pred, average='weighted'))

-------------
The score of the cats model is (TRAIN): 0.747475
-------------
The score of the cats model is (TEST): 0.739507
-------------
The best params for logistic regression on cats is: {'C': 0.001} 
-------------
The F1 score for the dogs model is (TEST): 0.628765


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
